In [29]:
!pip install pandas
!pip install openpyxl
!pip install yfiles_jupyter_graphs
!pip install networkx



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [30]:
import pandas as pd
import os
from yfiles_jupyter_graphs import GraphWidget


In [31]:
notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/berber_1.xlsx")

df = pd.read_excel(file_path, header=None)

In [32]:

# shows empty widget
w = GraphWidget()

w.orthogonal_layout()

In [61]:
#for (index, colname) in enumerate(df):
#  if index > 2:
#    print(index, df[colname].values[2])

EPOCH_ROW = 0
CYCLE_ROW = 1
CYCLE_RULE_ROW = 2
VISIT_LABEL_ROW = 3
CONDITION_ROW = 4
VISIT_WINDOW_ROW = 5
FIRST_ACTIVITY_ROW = 7

FIRST_VISIT_COL = 3

tp_index = 1
timepoints = []
visits = []
activities = []
activity_bc = {}
activity_row = []

activity = None
for index, col_def in df.iterrows():
  #print("ACT1:", index, df.iloc[index, 1])
  if index >= 7:
    activity_present = not pd.isnull(df.iloc[index, 1])
    if activity_present:
      activity = df.iloc[index, 1]
      activities.append(activity)
    activity_row.append({ 'name': activity, 'present': activity_present })
    bc_present = not pd.isnull(df.iloc[index, 2])
    if bc_present:
      if not activity in activity_bc:
        activity_bc[activity] = { 'bc': [] }  
      activity_bc[activity]['bc'].append(df.iloc[index, 2])
print("activities:", activities)
print("activity_bc:", activity_bc)
print("activity_row", activity_row)

for index in range(df.shape[1]):
  if index >= FIRST_VISIT_COL:
    anchor = False
    condition_type = None
    label = df.iloc[VISIT_LABEL_ROW, index]
    if pd.isnull(df.iloc[CONDITION_ROW,index]):
      condition_info = ""
      condition_type = "unknown"
      condition = ""
    else:
      condition_info = df.iloc[CONDITION_ROW, index]
      condition_parts = condition_info.split(":")
      #print ("CPs:", condition_parts)
      if condition_parts[0].upper() == "A":
        anchor = True
        condition_type = "anchor"
      if condition_parts[0].upper() == "P":
        condition_type = "previous"
      elif condition_parts[0].upper() == "N":
        condition_type = "next"
      if len(condition_parts) == 2:
        condition = condition_parts[1].strip()
      else:
        condition = ""
    #print("C:", condition_type)
    window = df.iloc[VISIT_WINDOW_ROW, index]
    window_is_null = pd.isnull(df.iloc[VISIT_WINDOW_ROW,index])
    if window_is_null:
      window = ""
    visits.append({ 'name': label, 'window': window })
    timepoints.append({ 'name': "tp_%s" % (tp_index), 'condition_type': condition_type, 'condition': condition, 'anchor': anchor})
    tp_index += 1
print("timepoints:", timepoints)
print("visits:", visits)

tp_activity = {}
for tp in timepoints:
  tp_name = tp['name']
  tp_activity[tp_name] = {}
  for activity in activities:
    tp_activity[tp_name][activity] = False
for index in range(df.shape[1]):
  if index >= FIRST_VISIT_COL:
    column = df.iloc[:, index]
    row = 0
    for x in column:
      if row >= FIRST_ACTIVITY_ROW:
        if not pd.isnull(x):
          #print("X ROW:", index, row, x)
          if x.upper() == "X":
            if activity_row[row - FIRST_ACTIVITY_ROW]['present']:
              activity = activity_row[row - FIRST_ACTIVITY_ROW]['name']
              tp = timepoints[index - FIRST_VISIT_COL]['name']
              tp_activity[tp][activity] = True
      row += 1
print("tp_activity:", tp_activity) 

activities: ['Informed Consent', 'Inclusion / Exclusion Criteria', 'Physical Examination']
activity_bc: {'Physical Examination': {'bc': ['PE 1', 'PE 2', 'Weight', 'Height']}}
activity_row [{'name': 'Informed Consent', 'present': True}, {'name': 'Inclusion / Exclusion Criteria', 'present': True}, {'name': 'Physical Examination', 'present': True}, {'name': 'Physical Examination', 'present': False}, {'name': 'Physical Examination', 'present': False}, {'name': 'Physical Examination', 'present': False}]
timepoints: [{'name': 'tp_1', 'condition_type': 'next', 'condition': '0..30 Days', 'anchor': False}, {'name': 'tp_2', 'condition_type': 'anchor', 'condition': '', 'anchor': True}, {'name': 'tp_3', 'condition_type': 'previous', 'condition': '+14 Days', 'anchor': False}, {'name': 'tp_4', 'condition_type': 'previous', 'condition': '+N Days', 'anchor': False}, {'name': 'tp_5', 'condition_type': 'previous', 'condition': '+14 Days', 'anchor': False}, {'name': 'tp_6', 'condition_type': 'previous', 

In [66]:
ENTRY_NODE_INDEX = 0
EXIT_NODE_INDEX = 99999

nodes = [
  {'id': ENTRY_NODE_INDEX, 'properties': {'label': 'Entry'}},
  {'id': EXIT_NODE_INDEX, 'properties': {'label': 'Exit'}}
]
edges = []
node_index = 1
edge_index = 0
prev_node = None
activity_nodes = {}
tp_nodes = {}

# Add base timeline, the timepoints
for tp in timepoints:
  if tp['anchor']:
    label = "%s : ANCHOR" % (tp['name'])
  else:
    label = tp['name']
  nodes.append({ 'id': node_index, 'properties': {'label': label} })
  tp_nodes[tp['name']] = node_index
  if prev_node == None:
    edges.append( {id: edge_index, 'start': ENTRY_NODE_INDEX, 'end': node_index, 'properties': {'label': "entry"}})
  else:
    edges.append( {id: edge_index, 'start': prev_node, 'end': node_index, 'properties': {'label': "next"}})
  prev_node = node_index
  node_index += 1
  edge_index += 1
edges.append( {id: edge_index, 'start': prev_node, 'end': EXIT_NODE_INDEX, 'properties': {'label': "exit"}})

# Add conditions
for index, tp in enumerate(timepoints):
  start_name = tp['name']
  start_index = tp_nodes[start_name]
  if tp['condition_type'] == "next":
    end_name = timepoints[index + 1]['name']
    end_index = tp_nodes[end_name]
    edges.append( {id: edge_index, 'start': start_index, 'end': end_index, 'properties': {'label': tp['condition']}})
  elif tp['condition_type'] == "previous":
    end_name = timepoints[index - 1]['name']
    end_index = tp_nodes[end_name]
    edges.append( {id: edge_index, 'start': start_index, 'end': end_index, 'properties': {'label': tp['condition']}})
  edge_index += 1

# Add links to visits
for index, tp in enumerate(timepoints):
  visit_index = node_index
  visit = visits[index]
  nodes.append({ 'id': visit_index, 'properties': { 'label': "%s, %s" % (visit['name'], visit['window']) }})
  node_index += 1
  edges.append( {id: edge_index, 'start': tp_nodes[tp['name']], 'end': visit_index, 'properties': {'label': "contains_visit"}})
  edge_index += 1

# Add links to activities (a fresh copy for each, not shared, makes the diagram better to understand)
for tp, value in tp_activity.items():
  for activity, required in value.items():
    if required:
      activity_index = node_index
      nodes.append({ 'id': activity_index, 'properties': { 'label': activity }})
      #activity_nodes[activity] = activity_index
      node_index += 1
      if activity in activity_bc:
        for bc in activity_bc[activity]['bc']:
          bc_index = node_index
          nodes.append({ 'id': bc_index, 'properties': { 'label': bc }})
          node_index += 1
          edges.append( {id: edge_index, 'start': activity_index, 'end': bc_index, 'properties': {'label': "contains_bc"}})
          edge_index += 1
      edges.append( {id: edge_index, 'start': tp_nodes[tp], 'end': activity_index, 'properties': {'label': "contains_activity"}})
      edge_index += 1

# Display
w.set_directed(True)
w.set_nodes(nodes)
w.set_edges(edges)
w

/Users/daveih/Library/Python/3.10/lib/python/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
keys must be str, int, float, bool or None, not builtin_function_or_method
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


GraphWidget(layout=Layout(height='500px', width='100%'))

In [35]:
a = w.get_graph_layout()
print(a)

{'algorithm': 'orthogonal', 'options': {}}
